In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.python.keras.optimizer_v2 import adam

In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

samples = 130967 # 2~130968
training_samples = 28317 *4 # 2~113269 (Training)
validation_samples = 3541 *4 # 113270~127433 (PublicTest)
test_samples = 3535 # 127434~130968 (PrivateTest)


In [3]:
# load data
image_path = "./dataset_f+r25/images.npy"
emotion_multi_path = "./dataset_f+r25/emotions_multi.npy"
emotion_single_path = "./dataset_f+r25/emotions_single.npy"

images = np.load(image_path)
emotions_multi = np.load(emotion_multi_path)
emotions_single = np.load(emotion_single_path)

print(images.shape)
print(emotions_multi.shape)
print(emotions_single.shape)

(130967, 48, 48, 1)
(130967, 8)
(130967, 8)


In [4]:
#images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(images))
images = tf.image.grayscale_to_rgb(layers.Rescaling(1./127.5, offset= -1)(tf.convert_to_tensor(images)))
emotions = tf.convert_to_tensor(emotions_multi)

training_size = training_samples + validation_samples
test_size = test_samples
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

print("training_emotions shape:", training_emotions.shape)
print("test_images shape:", test_images.shape)
print("test_emotions shape:", test_emotions.shape)

training_emotions shape: (127432, 8)
test_images shape: (3535, 48, 48, 3)
test_emotions shape: (3535, 8)


In [5]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [6]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    return model.fit(x=training_images,
                     y=training_emotions,
                     batch_size=batch_size,
                     epochs=num_epochs,
                     validation_data=(test_images, test_emotions))

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

model = create_model()

learning_rate = 1e-4
num_epochs = 40
loss = losses.MeanSquaredError()
batch_size = 32

history = train(model, learning_rate , loss, num_epochs, batch_size)

C:\Users\Darkl\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/40
3983/3983 [==============================] - 270s 66ms/step - loss: 0.0247 - model_acc: 0.7242 - val_loss: 0.0166 - val_model_acc: 0.7968
Epoch 2/40
 983/3983 [======>.......................] - ETA: 3:11 - loss: 0.0158 - model_acc: 0.8085

In [ ]:
import pickle

with open('history_base+flip+rotate25.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
'''
# best model
train_loss = history.history['loss']
val_loss   = history.history['val_loss']
train_acc  = history.history['model_acc']
val_acc    = history.history['val_model_acc']
epochs     = range(num_epochs)

fig, ax1 = plt.subplots()
ax1.set_xlabel('Epochs', fontsize=16)
ax1.set_ylabel('Loss', fontsize=16)
line1,=ax1.plot(epochs, train_loss, color='red')
line1.set_label('training loss')
line2,=ax1.plot(epochs, val_loss, color='blue')
line2.set_label('validation loss')
ax1.tick_params(axis='y')
ax1.legend()

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', fontsize=16) 
line3,=ax2.plot(epochs, train_acc, color='green', label='training accuracy')
line3.set_label('training accuracy')
line4,=ax2.plot(epochs, val_acc, color='darkviolet', label='validation accuracy')
line4.set_label('validation accuracy')
ax2.tick_params(axis='y')
ax1.legend()
ax2.legend(loc=4)

fig.tight_layout()
plt.show()
'''

In [ ]:
'''
# model comparison
val_acc1 = history1.history['val_model_acc']
val_acc2 = history2.history['val_model_acc']
val_acc3 = history3.history['val_model_acc']
epochs = range(num_epochs)

fig = plt.figure()
plt.plot(epochs, val_acc1, label = "line 1")
plt.plot(epochs, val_acc2, label = "line 2")
plt.plot(epochs, val_acc3, label = "line 3")
plt.legend()
plt.show()
'''

In [ ]:
'''
# save plot
save_plot_file_name = "model_comparison.png"
if not os.path.isdir('./results/'):
    os.mkdir('./results/')
fig.savefig(os.path.join('./results/', save_plot_file_name))
'''